# Deploying Machine Learning Models to Production


## Objective

To deploy the trained model as a web appication to Arundo Fabric and evaluate the performance of the deployed model on unseen data. 

## How?

Based on your trained model you will create a web application that you will then deploy to a cloud production environment (Arundo Fabric) using our [Arundo Composer](https://docs.arundo.com/composer/overview/composer-cli) tool. You'll then be able to interact with your applications either using the UI, [Model Manager](https://docs.arundo.com/fabric/models-manager) or [composer command-line interface](https://docs.arundo.com/composer/overview/composer-cli). Finally you'll be able to use the `grader` application to evaluate the model that you deployed in your application. 

## Requirements

A persisted and trained model.


### Note

It is possible to deploy multiple applications should you experiment with different models. 

# Model deployment instructions

Composer is a command like interface (CLI) tool that gives you the ability to publish machine learning models into the Arundo Fabric cloud environment. 

[Arundo Composer documentation](https://docs.arundo.com/composer/overview/composer-cli)

Arundo Composer is already available in your environment and you should be able to use it either from the notebooks (using the `!` to delegate execution to the shell) or starting a new terminal on JupyterHub. To start a terminal go to home and click `New` of the right hand side and select `Terminal` option.

## Configuring composer

To see the current configuration of Arundo Composer
```bash
composer check
```

We'll need to switch to the proper environment with:
```bash
composer switch eu-production
```

Now you can login with your email address and you'll be asked to provide a password that we sent over earlier.
```bash
composer login -e your.email@example.com
```

You can do the
```bash
composer check
```
again to verify the you're in the right environment and that the you're logged in, you should see a response similar to this:
```
Composer Version:   0.5.0
Email:              first.last+amld2019@arundo.com
Saved Password:     True
Server:             https://eu-enterprise.arundo.com
Access Remaining:   5h40m
Refresh Remaining:  29d23h
OS Version:         darwin
```

In [ ]:
# go ahead and try to log in with composer
!composer check

## Preparing for deployment

To deploy to the cloud, your application needs (at least) the following three files:
```
app.py
config.yaml
requirements.txt```


In fact `requirements.txt` is not even necessary if you have no additional Python library dependencies beyond the standard libraries. The following cells show the corresponding contents of these files for the model which you have already seen deployed.

# Turning a machine learning model into a web application

- [model workspace](https://docs.arundo.com/composer/model-workspaces)
- [tutorial with an example](https://docs.arundo.com/composer/tutorials/building-a-model)

### config.yaml

This file names the model and provides some basic version control. Note that in the tenant no two models can have the same name and version number regardless of which user deployed it. If you try to deploy a model whose name/version already exists, then the build will fail. To make sure your name does not clash with other workshop users we suggest to add your github user handle in the model name, this will also make it easier to find in the Models Management table when you have deployed to Fabric. If your build fails, fix the code and bump the version numer when redeploying.

Example of config.yaml file: 

```
model_name: OneClassSVM
model_version: 0.0.1
wrapper_version: 2
```

### requirements.txt

Here you should add all the Python library requirements needed to execute your model so that they are installed correctly in the deployed environment. The main cause of failures in the deployment process tends to be a missing entry in this file, particularly of hidden dependencies. If your model has associated non-python binary dependencies which would need e.g. `apt-get install <PACKAGE>` then you will need an extra file `dependencies.apt` which lists these. For most workflows that shouldn't be needed.

Example of a `requirements.txt` file: 

```
numpy==1.16.0
scikit-learn==0.20.2
```

### app.py

This file is where most of the changes take place. Here there is an endpoint implemented, the details of which are given in the docstrings for those endpoints.

In [ ]:
# example of app.py file

"""
To deploy your own serialized model

"""

from runtime.framework import endpoint, argument, returns
import numpy as np

# model dependent libraries
from sklearn.externals import joblib


# global variables can be loaded into memory to improve efficiency
# be careful if using tensorflow globally
# update file names according to your own models, as needed

scaler_filename = 'scaler.model'
model_filename = 'RandomForestClassifier.model'
G_SCALER = joblib.load(scaler_filename)
MODEL = joblib.load(model_filename)


# update the argument names below based on the final list of features
# that you have used in your model. 


@endpoint()
@argument("setting1", type=float, description="Value of input sensor setting1")
@argument("setting2", type=float, description="Value of input sensor setting2")
@argument("setting3", type=float, description="Value of input sensor setting3")
@argument("s1", type=float, description="Value of input sensor s1")
@argument("s2", type=float, description="Value of input sensor s2")
@argument("s3", type=float, description="Value of input sensor s3")
@argument("s4", type=float, description="Value of input sensor s4")
@argument("s5", type=float, description="Value of input sensor s5")
@argument("s6", type=float, description="Value of input sensor s6")
@argument("s7", type=float, description="Value of input sensor s7")
@argument("s8", type=float, description="Value of input sensor s8")
@argument("s9", type=float, description="Value of input sensor s9")
@argument("s10", type=float, description="Value of input sensor s10")
@argument("s11", type=float, description="Value of input sensor s11")
@argument("s12", type=float, description="Value of input sensor s12")
@argument("s13", type=float, description="Value of input sensor s13")
@argument("s14", type=float, description="Value of input sensor s14")
@argument("s15", type=float, description="Value of input sensor s15")
@argument("s16", type=float, description="Value of input sensor s16")
@argument("s17", type=float, description="Value of input sensor s17")
@argument("s18", type=float, description="Value of input sensor s18")
@argument("s19", type=float, description="Value of input sensor s19")
@argument("s20", type=float, description="Value of input sensor s20")
@argument("s21", type=float, description="Value of input sensor s21")
@returns("predicted_class", type=float,
         description="Prediction as a float: 1.0 if anomaly, 0.0 otherwise")
def anomaly_detection(
    setting1, setting2, setting3,
    s1,  s2,  s3,  s4,  s5,  s6,  s7,  s8,  s9,  s10,
    s11, s12, s13, s14, s15, s16, s17, s18, s19, s20, s21
    ):
    """
    This endpoint is used to connect a live stream of data to a model and
    return predictions in real time for each new feature row as it arrives
    from a remote location.
    """

    # take the inputs and put them into and ordered np.array shape (, 29)
    feature_list = np.array([
        setting1, setting2, setting3,
        s1,  s2,  s3,  s4,  s5,  s6,  s7,  s8,  s9,  s10,
        s11, s12, s13, s14, s15, s16, s17, s18, s19, s20, s21
    ])

    # apply the globally loaded feature scaler
    scaled = G_SCALER.transform(np.atleast_2d(feature_list))

    # perform prediction using the model
    yhat = MODEL.predict(scaled)

    return float(yhat)


## Testing before deployment

To test if `app.py` is working as expected we can run the model in the current environment before deploying. Note that if this was on your laptop you could also access the auto rendered web form by navigating to port 5000 on the local host. The virtual machine has not been setup to render the forms so we will instead check by sending a request from the command line to that localhost port.

First you will need to open a terminal from the notebook homepage, `cd work` and then issue the command `composer run`, if this fails you will need to debug `app.py`. Otherwise you should be able to execute the two following commands:

In [ ]:
!curl -d '@{../templates/test_row.json}' -H "Content-Type: application/json" http://localhost:5000/stream_prediction 

## Deploying to Arundo Fabric

Deployment is simple. Run the following in your terminal, after navigating to your app folder: `composer publish`

If the build or deploy fails, see if you can determine why or call one of the session conveners to help you debug. Otherwise you will now see the model listed as follows: `composer model list`

Further information about the deployed model can be obtained directly from composer, for example:

```
composer model details <MODEL_UUID>
composer model endpoints <MODEL_UUID>
```

## Application deployed for model evaluation

We have deployed an application, refered to as `grader`, that will execute the model you have deployed in your application, apply it to make prediction on unseen data and return the performance of your model in relation to all models that have been deployed. 

# Arundo Fabric

https://docs.arundo.com/fabric/overview


# Model Manager

https://docs.arundo.com/fabric/models-manager

# Evaluate performance of deployed model

Click on the following endpoint to use the `grader` application: 